In [1]:
import random

import numpy as np
import torch
from torch.utils.data.dataloader import default_collate

from settings import EXPERIMENTS_DIR
from experiment import Experiment
from utils2 import to_device, load_weights, load_embeddings, create_embeddings_matrix
from vocab import Vocab
from train2 import create_model
from preprocess import load_dataset, create_dataset_reader

In [2]:
import argparse
import json
from models import MLPClassifier, Baseline_Embeddings
from models import Seq2Seq, MLP_D, MLP_G, MLP_I, MLP_I_AE, JSDistance, Seq2SeqCAE, Baseline_Embeddings, Baseline_LSTM
from utils import to_gpu, Corpus, batchify, SNLIDataset, collate_snli
import random
import pickle as pkl
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


In [3]:
exp_id = 'train.3ir9y_e3'

# Load everything

In [4]:
EXPERIMENTS_DIR

PosixPath('data/experiments')

In [5]:
exp = Experiment.load(EXPERIMENTS_DIR, exp_id)

In [6]:
preprocess_exp = Experiment.load(EXPERIMENTS_DIR, exp.config.preprocess_exp_id)
dataset_train, dataset_val, dataset_test, vocab, style_vocab, W_emb = load_dataset(preprocess_exp)

Dataset: 453655, val: 10000, test: 10000
Vocab: 9419, style vocab: 2
W_emb: (9419, 300)


In [7]:
dataset_reader = create_dataset_reader(preprocess_exp.config)

In [8]:
model = create_model(exp.config, vocab, style_vocab, dataset_train.max_len, W_emb)

In [9]:
dataset_train.max_len

20

In [10]:
load_weights(model, exp.experiment_dir.joinpath('best.th'))

In [11]:
model = model.eval()

## Predict

In [12]:
def create_inputs(instances):
    if not isinstance(instances, list):
        instances = [instances,]
        
    if not isinstance(instances[0], dict):
        sentences = [
            dataset_reader.preprocess_sentence(dataset_reader.spacy( dataset_reader.clean_sentence(sent)))
            for sent in instances
        ]
        
        style = list(style_vocab.token2id.keys())[0]
        instances = [
            {
                'sentence': sent,
                'style': style,
            }
            for sent in sentences
        ]
#         print("INST!")
        for inst in instances:
#             print(inst)
            inst_encoded = dataset_train.encode_instance(inst)
            inst.update(inst_encoded)            
    
    
    instances = [
        {
            'sentence': inst['sentence_enc'],
            'style': inst['style_enc'],
        } 
        for inst in instances
    ]
    
    instances = default_collate(instances)
    instances = to_device(instances)      
    
    return instances

In [13]:
def get_sentences(outputs):
    predicted_indices = outputs["predictions"]
    end_idx = vocab[Vocab.END_TOKEN]
    
    if not isinstance(predicted_indices, np.ndarray):
        predicted_indices = predicted_indices.detach().cpu().numpy()

    all_predicted_tokens = []
    for indices in predicted_indices:
        indices = list(indices)

        # Collect indices till the first end_symbol
        if end_idx in indices:
            indices = indices[:indices.index(end_idx)]

        predicted_tokens = [vocab.id2token[x] for x in indices]
        all_predicted_tokens.append(predicted_tokens)
        
    return all_predicted_tokens

In [14]:
sentence =  ' '.join(dataset_val.instances[1]['sentence'])

In [15]:
sentence

'they are really good people .'

In [16]:
inputs = create_inputs(sentence)

In [17]:
inputs

{'sentence': tensor([[136,  55,  29, 380, 368,   8,   2,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0]], device='cuda:0'),
 'style': tensor([0], device='cuda:0')}

In [18]:
outputs = model(inputs)

In [19]:
sentences = get_sentences(outputs)

In [20]:
' '.join(sentences[0])

'they are really good people .'

### Swap style

In [21]:
possible_styles = list(style_vocab.token2id.keys()) #['negative', 'positive']

In [22]:
style_vocab.token2id.keys()

dict_keys(['negative', 'positive'])

In [23]:
possible_styles

['negative', 'positive']

In [24]:
sentences0 = [s for s in dataset_val.instances if s['style'] == possible_styles[0]]
sentences1 = [s for s in dataset_val.instances if s['style'] == possible_styles[1]]

In [25]:
for i in np.random.choice(np.arange(len(sentences0)), 5):
    print(i, ' '.join(sentences0[i]['sentence']))

3190 after the discount , they were $ number .
3586 anyway , you order your food to the bartenders .
1882 i mean really , really bad .
2197 apparently the guy was the owner , but he was beyond rude .
1968 neither of them seemed to want to help me .


In [26]:
for i in np.random.choice(np.arange(len(sentences1)), 5):
    print(i, ' '.join(sentences1[i]['sentence']))

2450 we started out with a cheese tasting , which was very good .
796 it is pricey , but the food is of high quality and worth it .
5455 really good pizza .
3798 great experience with this office !
1454 my skin looks and feels great and she knew exactly what my skin needed .


#### Swap

In [27]:
print(3874, ' '.join(sentences0[3874]['sentence']))

3874 the rice had hard things in it .


In [28]:
target0 = 3874 # np.random.choice(np.arange(len(sentences0)))
target1 = 4935 # np.random.choice(np.arange(len(sentences0)))

In [29]:
print(' '.join(sentences0[target0]['sentence']))

the rice had hard things in it .


In [30]:
print(' '.join(sentences1[target1]['sentence']))

which is awesome !


In [31]:
inputs = create_inputs([
    sentences0[target0],
    sentences1[target1],
])

In [32]:
z_hidden = model(inputs)

In [33]:
z_hidden

{'decoder_hidden': tensor([[ 0.9960, -0.1104,  0.3478,  0.1852,  0.5692, -0.3471,  0.3468, -0.1282,
          -0.1176,  0.3602,  0.2054, -0.6964,  0.2084, -0.1360, -0.1278, -0.0855,
          -0.5349,  0.5455, -0.1412,  0.2628,  0.3365, -0.2249,  0.4589,  0.1841,
          -0.0585,  0.5467, -0.3516,  0.9126, -0.3469, -0.8066, -0.6026, -0.4069,
          -0.2361, -0.3126, -0.8887, -0.6321, -0.7467,  0.3509, -0.0832,  0.5541,
          -0.2939,  0.9630,  0.3044, -0.6743, -0.1846,  0.0799, -0.7861, -0.0899,
          -0.0869, -0.1461, -1.2180, -0.1254,  0.5410, -0.2104, -1.0996,  0.2980,
          -0.0926, -0.2994, -0.6431, -0.2990,  0.2858, -0.9104,  0.0771, -0.2561,
           0.0206,  0.7998,  0.6394, -0.2721,  0.2689, -0.5429, -0.7816, -0.6238,
          -0.0152,  0.9413, -0.2934, -0.5155,  0.8586,  0.4707, -0.7585,  0.4288,
           0.6684,  0.1305,  0.7862,  0.0903,  0.5864, -0.6920, -0.1550, -0.9478,
          -0.7752,  0.2467, -0.3270, -0.7186, -0.0455,  0.3155,  0.2718, -0.4417

In [81]:
z_hidden['style_hidden']

tensor([[ 9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.

In [35]:
z_hidden['meaning_hidden'].shape

torch.Size([2, 300])

In [123]:
original_decoded

{'decoder_hidden': tensor([[ 0.9960, -0.1104,  0.3478,  0.1852,  0.5692, -0.3471,  0.3468, -0.1282,
          -0.1176,  0.3602,  0.2054, -0.6964,  0.2084, -0.1360, -0.1278, -0.0855,
          -0.5349,  0.5455, -0.1412,  0.2628,  0.3365, -0.2249,  0.4589,  0.1841,
          -0.0585,  0.5467, -0.3516,  0.9126, -0.3469, -0.8066, -0.6026, -0.4069,
          -0.2361, -0.3126, -0.8887, -0.6321, -0.7467,  0.3509, -0.0832,  0.5541,
          -0.2939,  0.9630,  0.3044, -0.6743, -0.1846,  0.0799, -0.7861, -0.0899,
          -0.0869, -0.1461, -1.2180, -0.1254,  0.5410, -0.2104, -1.0996,  0.2980,
          -0.0926, -0.2994, -0.6431, -0.2990,  0.2858, -0.9104,  0.0771, -0.2561,
           0.0206,  0.7998,  0.6394, -0.2721,  0.2689, -0.5429, -0.7816, -0.6238,
          -0.0152,  0.9413, -0.2934, -0.5155,  0.8586,  0.4707, -0.7585,  0.4288,
           0.6684,  0.1305,  0.7862,  0.0903,  0.5864, -0.6920, -0.1550, -0.9478,
          -0.7752,  0.2467, -0.3270, -0.7186, -0.0455,  0.3155,  0.2718, -0.4417

In [36]:
original_decoded = model.decode(z_hidden)

In [133]:
original_decoded

{'decoder_hidden': tensor([[ 0.9960, -0.1104,  0.3478,  0.1852,  0.5692, -0.3471,  0.3468, -0.1282,
          -0.1176,  0.3602,  0.2054, -0.6964,  0.2084, -0.1360, -0.1278, -0.0855,
          -0.5349,  0.5455, -0.1412,  0.2628,  0.3365, -0.2249,  0.4589,  0.1841,
          -0.0585,  0.5467, -0.3516,  0.9126, -0.3469, -0.8066, -0.6026, -0.4069,
          -0.2361, -0.3126, -0.8887, -0.6321, -0.7467,  0.3509, -0.0832,  0.5541,
          -0.2939,  0.9630,  0.3044, -0.6743, -0.1846,  0.0799, -0.7861, -0.0899,
          -0.0869, -0.1461, -1.2180, -0.1254,  0.5410, -0.2104, -1.0996,  0.2980,
          -0.0926, -0.2994, -0.6431, -0.2990,  0.2858, -0.9104,  0.0771, -0.2561,
           0.0206,  0.7998,  0.6394, -0.2721,  0.2689, -0.5429, -0.7816, -0.6238,
          -0.0152,  0.9413, -0.2934, -0.5155,  0.8586,  0.4707, -0.7585,  0.4288,
           0.6684,  0.1305,  0.7862,  0.0903,  0.5864, -0.6920, -0.1550, -0.9478,
          -0.7752,  0.2467, -0.3270, -0.7186, -0.0455,  0.3155,  0.2718, -0.4417

In [37]:
original_sentences = get_sentences(original_decoded)

In [38]:
print(' '.join(original_sentences[0]))
print(' '.join(original_sentences[1]))

the rice was hard things that .
which is awesome !


In [39]:
test = model(create_inputs('sophistication extraordinary asdsadasdadsadasdsadasdsadasdadsadsd <unk>'))

In [40]:
get_sentences(model.decode(test))

[['great', 'for', 'a', 'good', 'food', 'at', 'what', '!']]

In [41]:
z_hidden['meaning_hidden'][1]

tensor([ 0.1151, -0.1436, -0.0410, -0.0718,  0.0400, -0.2186, -0.0190, -0.1394,
         0.1942,  0.0577,  0.0691,  0.1039,  0.0104,  0.1910,  0.2270,  0.0861,
        -0.0114, -0.1513,  0.0916,  0.1948,  0.4662,  0.2228, -0.1103, -0.3451,
         0.1512,  0.1191, -0.2565,  0.0299,  0.1555,  0.0458,  0.0703,  0.1774,
         0.0896, -0.1395, -0.2851,  0.0598, -0.0616,  0.3513, -0.0849, -0.0222,
        -0.1975,  0.2362, -0.0310,  0.2208,  0.1290, -0.0931,  0.0319,  0.0181,
        -0.0430, -0.0250,  0.0980,  0.2651,  0.1079, -0.0962,  0.1475,  0.0539,
        -0.0270,  0.0803, -0.0017,  0.0375, -0.0398,  0.0814,  0.0573,  0.1632,
        -0.0345,  0.0251, -0.1820, -0.0183,  0.3719, -0.1480, -0.0779,  0.0188,
        -0.1145, -0.1644,  0.0566,  0.1003,  0.1201, -0.1276,  0.0022,  0.0692,
        -0.0412,  0.1950,  0.1714,  0.1448, -0.2862, -0.1194, -0.2141,  0.0844,
         0.1872,  0.0271,  0.2977, -0.0764,  0.0421, -0.0412, -0.1529, -0.0319,
         0.0711, -0.0395, -0.2343, -0.14

In [42]:
for i in range (0, z_hidden['style_hidden'][0].shape[0]):
    z_hidden['style_hidden'][0][i] = 0.9


In [43]:
z_hidden['style_hidden'][0].shape[0]

300

In [44]:
z_hidden['style_hidden']

tensor([[ 9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,  9.0000e-01,
          9.0000e-01,  9.0000e-01,  9.

In [45]:
z_hidden['style_hidden'][0]

tensor([0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 0.9000, 0.9000, 0.9000, 

In [46]:
z_hidden_swapped = {
    'meaning_hidden': torch.stack([
        z_hidden['meaning_hidden'][0].clone(),
        #z_hidden['meaning_hidden'][1].clone(),        
    ], dim=0),
    'style_hidden': torch.stack([
        z_hidden['style_hidden'][0].clone(),
        #z_hidden['style_hidden'][0].clone(),        
    ], dim=0),
}

In [47]:
swaped_decoded = model.decode(z_hidden_swapped)

In [48]:
swaped_sentences = get_sentences(swaped_decoded)

In [49]:
print(' '.join(original_sentences[0]))
# print(' '.join(original_sentences[1]))
print()
print(' '.join(swaped_sentences[0]))
# print(' '.join(swaped_sentences[1]))

the rice was hard things that .

fish fresh and were delicious !


In [50]:
# EVERYTHING USEFUL IS BELOW

In [51]:
#python3.6 train_surrogate.py --data_path ./data/classifier --save_path game_output/ --classifier_path ./data --load_pretrained .
cur_dir = '.'

with open(cur_dir + '/vocab.json', 'r') as fin:
    corpus_vocab = json.load(fin)

corpus_train = SNLIDataset(train=True, vocab_size=11004-4, path='./data/classifier')
corpus_test = SNLIDataset(train=False, vocab_size=11004-4, path='./data/classifier')
trainloader= torch.utils.data.DataLoader(corpus_train, batch_size = 32, collate_fn=collate_snli, shuffle=True)
train_iter = iter(trainloader)
testloader= torch.utils.data.DataLoader(corpus_test, batch_size = 32, collate_fn=collate_snli, shuffle=False)
random.seed(1111)
np.random.seed(1111)
torch.manual_seed(1111)

EPS = 3e-2


original vocab 41574; pruned to 11004
Number of sentences dropped from ./data/classifier/train.txt: 448221 out of 549367 total
original vocab 41574; pruned to 11004
Number of sentences dropped from ./data/classifier/test.txt: 8288 out of 9824 total


In [111]:
trainloader.__dict__.keys()

dict_keys(['dataset', 'batch_size', 'num_workers', 'collate_fn', 'pin_memory', 'drop_last', 'timeout', 'worker_init_fn', 'sampler', 'batch_sampler', '_DataLoader__initialized'])

In [116]:
trainloader

In [52]:
autoencoder = torch.load(open(cur_dir + '/models/autoencoder_model.pt', 'rb'))
#gan_gen = torch.load(open(cur_dir + '/models/gan_gen_model.pt', 'rb'))
#gan_disc = torch.load(open(cur_dir + '/models/gan_disc_model.pt', 'rb'))
inverter = torch.load(open(cur_dir + '/models/inverter_model.pt', 'rb'))

classifier1 = Baseline_Embeddings(100, vocab_size=11004)
#classifier1 = Baseline_LSTM(100,300,maxlen=args.maxlen, gpu=args.cuda)
classifier1.load_state_dict(torch.load('./models' + "/baseline/model_emb.pt"))
vocab_classifier1 = pkl.load(open('./models' + "/vocab.pkl", 'rb'))

mlp_classifier = MLPClassifier(100 * 2, 3, layers='100-50')
#if not args.train_mode:
mlp_classifier.load_state_dict(torch.load('./game_output'+'/surrogate{0}.pt'.format('100-50')))

print(classifier1)
print(autoencoder)
print(inverter)
print(mlp_classifier)

optimizer = optim.Adam(mlp_classifier.parameters(),
                           lr=1e03,
                           betas=(0.9, 0.999))


/home/game/.local/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'models.Seq2SeqCAE' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Baseline_Embeddings(
  (embedding_prem): Embedding(11004, 100)
  (embedding_hypo): Embedding(11004, 100)
  (linear): Linear(in_features=200, out_features=3, bias=True)
)
Seq2SeqCAE(
  (embedding): Embedding(11004, 300)
  (embedding_decoder): Embedding(11004, 300)
  (encoder): Sequential(
    (layer-1): Conv1d(300, 500, kernel_size=(3,), stride=(1,))
    (bn-1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation-1): LeakyReLU(negative_slope=0.2, inplace)
    (layer-2): Conv1d(500, 700, kernel_size=(3,), stride=(2,))
    (bn-2): BatchNorm1d(700, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation-2): LeakyReLU(negative_slope=0.2, inplace)
    (layer-3): Conv1d(700, 1000, kernel_size=(3,), stride=(2,))
    (bn-3): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation-3): LeakyReLU(negative_slope=0.2, inplace)
  )
  (linear): Linear(in_features=1000, out_features=300, bias=

In [53]:
from torch.autograd import Variable

def evaluate_model():
    classifier1.eval()

    test_iter = iter(trainloader)
    correct=0
    total=0
    for batch in test_iter:
        premise, hypothesis, target, _, _, _, _ = batch

        if args.cuda:
            premise=premise.cuda()
            hypothesis = hypothesis.cuda()
            target = target.cuda()

        prob_distrib = classifier1.forward((premise, hypothesis))
        predictions = np.argmax(prob_distrib.data.cpu().numpy(), 1)
        correct+=len(np.where(target.data.cpu().numpy()==predictions)[0])
        total+=premise.size(0)
    acc=correct/float(total)
    print("Accuracy:{0}".format(acc))
    return acc

autoencoder.gpu = True
autoencoder = autoencoder.cuda()
autoencoder.start_symbols = autoencoder.start_symbols.cuda()
#gan_gen = gan_gen.cuda()
#gan_disc = gan_disc.cuda()
classifier1 = classifier1.cuda()
inverter = inverter.cuda()
mlp_classifier = mlp_classifier.cuda()


In [54]:
def train_process(premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    #mx = target.max().item()
    #assert(mx >= 0 and mx < 3)
    #for s, s_w in zip(premise, premise_words):
    #    for i, w in zip(s, s_w):
    #        assert(corpus_vocab.get(w, 3) == i)
    #print(hypothesis_words, flush=True)
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.train()

    #print(premise.max().item(), flush=True)
    #print(hypothesis.max().item(), flush=True)

    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()

    c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
    z_prem = inverter(c_prem).detach()

    c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False)
    z_hypo = inverter(c_hypo).detach()

    # z_comb = nn.cat((z_prem, z_hypo), 0).detach()

    output = mlp_classifier(z_prem, z_hypo)
    gold = classifier1((premise, hypothesis)).detach()

    #print(output.shape, flush=True)
    #print(gold.shape, flush=True)

    acc = (torch.argmax(gold, 1) == target).to(torch.float32).mean().item()
    acc_surrogate = (torch.argmax(output, 1) == target).to(torch.float32).mean().item()


    loss = -torch.mean(torch.sum(output * F.softmax(gold, dim=1), 1), 0)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), acc, acc_surrogate


In [144]:
def perturb(criterion, premise, hypothesis, target, premise_words, hypothesis_words, premise_length, hypothesis_length):
    autoencoder.eval()
    inverter.eval()
    classifier1.eval()
    mlp_classifier.eval()
    model.eval()
    
    premise_words = [premise_words]
    hypothesis_words = [hypothesis_words]
    premise_length = [premise_length]
    hypothesis_length = [hypothesis_length]


    premise_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in premise_words]).cuda()
    hypothesis_idx = torch.tensor([[corpus_vocab.get(w, 3) for w in s] for s in hypothesis_words]).cuda()
#     print('premise_idx = ' + str(premise_idx))
#     print('hypothesis-idx = ' + str(hypothesis_idx))
#     c_prem = autoencoder.encode(premise_idx, premise_length, noise=False)
#     print("OLD C_PREM")
#     print(c_prem.shape)
#     print(premise_words[0])
    c_prem_full = model(create_inputs(' '.join(premise_words[0])))
    c_prem = c_prem_full['style_hidden']
#     print("C_PREM")
#     print(c_prem)
#     print("----")
#     c_prem = create_inputs(' '.join(premise_words[0]))
    z_prem = inverter(c_prem).detach()
#     print('c_prem = ' + str(c_prem))
#     print('z_prem = ' + str(z_prem))
#     c_hypo = autoencoder.encode(hypothesis_idx, hypothesis_length, noise=False).detach()
    c_hypo_full = model(create_inputs(' '.join(hypothesis_words[0])))
    c_hypo = c_hypo_full['style_hidden'].detach()
#     print("C_HYPO")
#     print(c_hypo)
#     print("------")
    c_hypo.requires_grad = True
    z_hypo = inverter(c_hypo)


    premise = premise.unsqueeze(0)
    hypothesis = hypothesis.unsqueeze(0)
    target = target.unsqueeze(0)

    output = mlp_classifier(z_prem, z_hypo)
#     print("OUTPUT")
#     print(output)

    loss = criterion(output, target)
    mlp_classifier.zero_grad()
    inverter.zero_grad()
    loss.backward()

    direction = torch.sign(c_hypo.grad)
    nc_hypo = c_hypo + EPS * direction
#     nhypo_idx = autoencoder.generate(nc_hypo, 10, False)
    c_prem_full['style_hidden'] = nc_hypo
#     print(c_prem_full)
    
    
#     nhypo_idx = model.decode(nc_hypo)
    nhypo_idx = model.decode(c_prem_full)
#     return nhypo_idx.squeeze(0).cpu().numpy()
    return nhypo_idx


In [56]:
def classifier_pred(pw, hw):
    classifier1.eval()

    premise_idx = torch.tensor([vocab_classifier1.get(w, 3) for w in pw]).cuda().unsqueeze(0)
    hypothesis_idx = torch.tensor([vocab_classifier1.get(w, 3) for w in hw]).cuda().unsqueeze(0)

    return F.softmax(classifier1((premise_idx, hypothesis_idx)), 1).squeeze(0).cpu().detach().numpy()


In [57]:
def maximum(arr):
    a = -1
    t = 0
    for i in range(len(arr)):
        if arr[i] > a:
            a = arr[i]
            t = i
    return t


In [58]:
Vocab.UNK_TOKEN

'<unk>'

In [148]:
criterion = nn.CrossEntropyLoss().cuda()

niter = 0
evaluate = 0
nevaluate = 0
idx2words = dict(map(lambda x: (x[1], x[0]), corpus_vocab.items()))
while niter < len(testloader):
    niter += 1
    batch = train_iter.next()
    for p, h, t, pw, hw, pl, hl in zip(*batch):
        nh = perturb(criterion, p.cuda(), h.cuda(), t.cuda(), pw, hw, pl, hl)
        print('--------------------------------')
        print('Target ', t)
        print(' '.join(pw))
        print(' '.join(hw))
#         nhw = (['<sos>'] + [idx2words[i] for i in nh])[:10]
        nhw = '<sos> ' + ' '.join(get_sentences(model.decode(nh))[0])
        print(nhw)
        print('Old ', classifier_pred(pw, hw))
        print('New ', classifier_pred(pw, nhw))
        print('Old Pred: ', maximum(classifier_pred(pw, hw)))
        print('New Pred: ', maximum(classifier_pred(pw, nhw)))
        print('Good: ', maximum(classifier_pred(pw, hw)) != maximum(classifier_pred(pw, nhw)))
        evaluate = evaluate + int(maximum(classifier_pred(pw, hw)) != maximum(classifier_pred(pw, nhw)))
        nevaluate = nevaluate + 1
    break
print(evaluate / nevaluate)
print(nevaluate)



--------------------------------
Target  tensor(0)
<sos> several people are dancing in some sand . <eos>
<sos> the people are at the beach . <pad> <pad>
<sos> lots lots lots lots in lots out of spring rolls !
Old  [0.22600839 0.36014894 0.41384265]
New  [0.12953717 0.48788851 0.3825743 ]
Old Pred:  2
New Pred:  1
Good:  True
--------------------------------
Target  tensor(2)
<sos> a red-haired girl holds a cute baby goat .
<sos> a girl is holding a small dog . <pad>
<sos> lots lots of live in for watch games in fountain !
Old  [0.5548856  0.18146656 0.2636478 ]
New  [0.18848984 0.5135597  0.29795048]
Old Pred:  0
New Pred:  1
Good:  True
--------------------------------
Target  tensor(0)
<sos> an older lady wearing a tinkerbell jacket . <eos>
<sos> a woman is wearing a jacket . <pad> <pad>
<sos> wide variety of take out in the corner and take out !
Old  [0.40246424 0.26268488 0.3348509 ]
New  [0.14847809 0.5076462  0.34387574]
Old Pred:  0
New Pred:  1
Good:  True
---------------------

--------------------------------
Target  tensor(1)
<sos> a plane is <oov> next to some water <eos>
<sos> the plane is on a runway . <pad> <pad>
<sos> wide variety take out of town and you in in out !
Old  [0.2680588  0.26103124 0.47091   ]
New  [0.15933205 0.49197853 0.3486895 ]
Old Pred:  2
New Pred:  1
Good:  True
--------------------------------
Target  tensor(2)
<sos> a child is holding an apple in his mouth
<sos> a kid is watching tv . <pad> <pad> <pad>
<sos> reasonable for take out of a seat and you out !
Old  [0.31187543 0.21894163 0.46918294]
New  [0.19414404 0.42774403 0.378112  ]
Old Pred:  2
New Pred:  1
Good:  True
--------------------------------
Target  tensor(1)
<sos> hang glider on beach <eos> <pad> <pad> <pad> <pad>
<sos> glider rider on beach <pad> <pad> <pad> <pad> <pad>
<sos> good prices for lots on on spring spring spring !
Old  [0.50757945 0.19229233 0.3001282 ]
New  [0.1776621  0.5130726  0.30926526]
Old Pred:  0
New Pred:  1
Good:  True
-------------------------